In [31]:
import glob
import pandas as pd
import os
import numpy as np

save = False
target_region = 'padgroup'
csv_folder = 'padgroup/v1.22.6lut1/padgroupv1.22.6whitelut0csv' # padgroup1.15hrdTGtestcsv  padgroup1.15csv padgroup1.15whitecsv
# csv_folder = 'padgroup/v1.15TG/padgroup1.15tp1csv'

src_folder = f'./models/checkpoints/{csv_folder}' #    padgroupv1.12cp2rgbcsv  

dst_folder = f'./models/{target_region}_csv'
if target_region == 'component':
    threshold_dict = {0.5: 0.5, 0.7: 1.0}
elif target_region == 'body':  # body用的0.5阈值
    threshold_dict = {0.5: 1, 0.8: 0}
else:
    threshold_dict = {0.5: 0.5, 0.8: 1.0}
# else:
#     threshold_dict = {0.5: 0.5, 0.6: 1.0, 0.65: 1.0, 0.7: 1.0, 0.75: 1.0, 0.8: 1.0}
all_res_list = []
for res_path in glob.glob(os.path.join(src_folder, f'{target_region}*.csv')):
#     if 'jiraissues' in res_path:
#         continue
    res_df = pd.read_csv(res_path, index_col=0)
    if 'okval_' in res_path.split('/')[-1]:
        continue
    print(res_path.split('/')[-1], len(res_df))
    res_df['dataset'] = res_path.split('/')[-1]
    
    if target_region == 'component':
        res_df = res_df.fillna(0)
    all_res_list.append(res_df)

all_res_df = pd.concat(all_res_list).reset_index(drop=True)
if save:
    all_res_df.to_csv(os.path.join(dst_folder, f'{target_region}_2404193.csv'))

padgroup_bbtest_padgroupv1.22.6lut1select_128128_lutp0_all_white_png_augimgFalse_sharp4_calibration_T1.0_threshold0.8.csv 1
padgroup_newval_padgroupv1.22.6lut1select_128128_lutp0_all_white_png_augimgFalse_sharp4_calibration_T1.0_threshold0.8.csv 1
padgroup_oldval_padgroupv1.22.6lut1select_128128_lutp0_all_white_png_augimgFalse_sharp4_calibration_T1.0_threshold0.8.csv 1
padgroup_njzh_padgroupv1.22.6lut1select_128128_lutp0_all_white_png_augimgFalse_sharp4_calibration_T1.0_threshold0.8.csv 1
padgroup_masked_padgroupv1.22.6lut1select_128128_lutp0_all_white_png_augimgFalse_sharp4_calibration_T1.0_threshold0.8.csv 1
padgroup_cur_jiraissue_padgroupv1.22.6lut1select_128128_lutp0_all_white_png_augimgFalse_sharp4_calibration_T1.0_threshold0.8.csv 1
padgroup_alltestval_padgroupv1.22.6lut1select_128128_lutp0_all_white_png_augimgFalse_sharp4_calibration_T1.0_threshold0.8.csv 1
padgroup_graphite_padgroupv1.22.6lut1select_128128_lutp0_all_white_png_augimgFalse_sharp4_calibration_T1.0_threshold0.8.csv

In [33]:
accuracy_cols = ['binary_accuracy', 'mclass_accuracy']
binary_f1score_cols = ['binary_weighted_f1_score']
ng_f1score_cols = ['mc_ng_f1_score']
ok_f1score_cols = ['mc_ok_f1_score']
binary_okf1score_cols = ['bi_ok_f1_score']
overall_score_list = []
for i, model_res in all_res_df.iterrows():
    accuracies = 0
    binary_f1scores = 0
    ng_f1scores = 0
    ok_f1scores = 0
    binary_okf1scores = 0
    
    for th, th_w in threshold_dict.items():
#         th_w = threshold_dict[th]
        accuracies += th_w*np.sum([model_res[f'{a}-th{th}'] if not np.isnan(model_res[f'{a}-th{th}']) else th_w for a in accuracy_cols])
        binary_f1scores += th_w*np.sum([model_res[f'{a}-th{th}'] if not np.isnan(model_res[f'{a}-th{th}']) else th_w for a in binary_f1score_cols])
        ng_f1scores += th_w*np.sum([model_res[f'{a}-th{th}'] if not np.isnan(model_res[f'{a}-th{th}']) else th_w for a in ng_f1score_cols])
        ok_f1scores += th_w*np.sum([model_res[f'{a}-th{th}'] if not np.isnan(model_res[f'{a}-th{th}']) else th_w for a in ok_f1score_cols])
        binary_okf1scores += th_w*np.sum([model_res[f'{a}-th{th}'] if not np.isnan(model_res[f'{a}-th{th}']) else th_w for a in binary_okf1score_cols])
        
    overall_score = accuracies*0.8 + binary_f1scores*1.0 + binary_okf1scores*0.8 + ng_f1scores*1.0 + ok_f1scores*0.8
#     overall_score = binary_okf1scores*1.0 + ok_f1scores*1.0
    overall_score_list.append(overall_score)

all_res_df['overall_score'] = overall_score_list
if save:
    all_res_df.to_csv(os.path.join(dst_folder, f'{target_region}_2404193.csv'))


In [34]:

if target_region == 'singlepad':
    valdataset_weights = {'newval': 0.5, 'bbtest': 0.3, 'bbtestmz':1.0, 
                          'alltestval': 0.0, 'jiraissues': 0.4, 'cur_jiraissue': 0.2} #  
    # valdataset_weights = {'bbtestmz':1.0}

#     valdataset_weights = {'alltestval': 0.0, 'newval': 0.0, 'bbtest1': 0.0, 'bbtest2':1.0, 'jiraissues': 0.3}

if target_region == 'singlepinpad':
#     valdataset_weights = {'alltestval': 0.0, 'newval': 0.0, 'bbtest': 0.0, 'bbtestmz':1.0, 'jiraissues': 0.0}
    valdataset_weights = {'alltestval': 0.5, 'newval': 0.3, 'bbtest': 0.3, 'bbtestmz':1.0, 'jiraissues': 0.2, 'cur_jiraissue': 0.1}
    
elif target_region == 'component':
    valdataset_weights = {'alltestval': 0.8, 'newval': 1, 'bbtestmz': 1.5, 'bbtest': 0.8, 'jiraissues': 0.2, 'tmpt': 0.5}

elif target_region == 'body':
    valdataset_weights = {'alltestval': 0.5, 'newval': 0.7, 'bbtest': 0.7, 'bbtestmz': 1, 'jiraissues': 0.5, 'cur_jiraissue': 0.2}  #, 'led_cross_pair': 1 , 'melf': 1
    # valdataset_weights = {'alltestval': 0.8, 'newval': 1, 'bbtest': 0.7, 'bbtestmz': 1, 'jiraissues': 0.1}  #, 'led_cross_pair': 0.3

#     valdataset_weights = {'bbtestmz': 1, 'bbtest': 0.4, 'jiraissues': 0.1}

elif target_region == 'padgroup':
    valdataset_weights = {'alltestval': 0.8, 'newval': 1, 'bbtest': 1, 'oldval': 1,
                           'jiraissues': 0.5, 'masked': 0.5, 'cur_jiraissue': 0.3} 
#, 'cur_jiraissue': 0.3, 'cur_jiraissue_26MHZ': 0.1, ,'cur_jiraissue': 0.1, 'cur_jiraissue_26MHZ': 0.1
# valdataset_weights = {a: b for a,b in zip(['alltestval', 'bbtest', 'oldval', 'masked'], 'cur_jiraissue_26MHZ': 0.1,, 'cur_jiraissue_26MHZ': 0.1
#                    

compare_res_list = [] 
nan_res_list = []
for group_key, group_df in all_res_df.groupby(['model_name', 'version_folder']):
    accumulated_score = 0
    for j, group_anno_j in group_df.iterrows():
        dataset_name = group_anno_j['val_dataset']
        if dataset_name not in valdataset_weights:
            continue
        # if 'v0.7' in group_anno_j['dataset']:
            # print(0)
            # pass
        if np.isnan(group_anno_j['overall_score']):
            print(f"{dataset_name} overall_score is nan")
            accumulated_score += valdataset_weights[dataset_name] * 0
            if group_df.loc[j, 'model_name'] not in nan_res_list:
                nan_res_list.append(group_df.loc[j, 'model_name'])
            # group_df.loc[j, 'version_folder'] = group_anno_j['version_folder'] +'_nan'
        else:
            accumulated_score += valdataset_weights[dataset_name]*group_anno_j['overall_score']
        
    model_res_df = group_df[['model_name', 'version_folder', 'backbone', 'region', 'ckp_epoch']].drop_duplicates()
    model_res_df['final_score'] = accumulated_score
    compare_res_list.append(model_res_df)

compare_res_df = pd.concat(compare_res_list).reset_index(drop=True)
if save:
    compare_res_df.to_csv(os.path.join(dst_folder, f'{target_region}_modelres_240412.csv'))

compare_res_df_sorted = compare_res_df.sort_values(by=['final_score'], ascending=False)
# compare_res_df_sorted[compare_res_df_sorted['version_folder'] == 'v0.2.2']

for model in list(compare_res_df_sorted['model_name']):
    print(model)
compare_res_df_sorted

padgroupfcdropoutmobilenetv3largers128128s42c2val0.1b256_ckp_bestv1.22.6slimmaskedf1certainlut1ed20.0j0.4lr0.025nb128nm128dual2top0.pth.tar


,model_name,version_folder,backbone,region,ckp_epoch,final_score
0,padgroupfcdropoutmobilenetv3largers128128s42c2...,padgroupv1.22.6lut1select,fcdropoutmobilenetv3large,padgroup,47,36.677977


In [36]:
# general datasets

##### ['bbtest', 'newval', 'alltestval']
if target_region == 'singlepinpad':
    dataset_cur_thres = {
        'alltestval':{
            'binary accuracy': 0.98448,   
            'mclass accuracy': 0.98328,
            },
        'newval': {
            'binary accuracy': 0.98818,
            'mclass accuracy': 0.98718,
        },
        'bbtest':{
            'binary accuracy': 0.97977,
            'mclass accuracy': 0.97962,            
        },
        'oldval': {
            'binary accuracy': 0.99953,
            'mclass accuracy': 0.99953,    
        },
        'jiraissues':{
            'binary accuracy': 0.99074, 
            'mclass accuracy': 0.99030,  
        },
        'cur_jiraissue':{
            'binary accuracy': 0.73913,
            'mclass accuracy': 0.73913, 
        },
    }
#     valdataset_weights = {'alltestval': 0.6, 'newval': 0.6, 'bbtest1': 0.5, 'bbtest2':1.0, 'jiraissues': 0.5}
    dataset_list = ['alltestval', 'newval', 'bbtest',  'bbtestmz', 'jiraissues','3D',  'cur_jiraissue', 'szfl', 'yl_cold_solder_0512', 'yl_cold_solder_TG', 
                    
                    ]
    lower_limit_range = 0.001
    test_dataset_cur_thres = {
        'bbtestmz':{
            'binary accuracy': 0.93096,
            'mclass accuracy': 0.92518,        
        },
        'yl_cold_solder_0512':{
            'binary accuracy': 0.99687,
            'mclass accuracy': 0.99687, 
        },
        'yl_cold_solder_TG':{
            'binary accuracy': 0.96000,
            'mclass accuracy': 0.92000, 
        },
        '3D':{
            'binary accuracy': 0.94397,
            'mclass accuracy': 0.93922, 
        },
        'szfl':{
            'binary accuracy': 1.00000,
            'mclass accuracy': 1.00000, 
        },

    }
    del_general_dataset_keys = []

    # valdataset_weights = {'alltestval': 0.5, 'newval': 0.3, 'bbtest1': 0.3, 'bbtestmz':1.0, 'jiraissues': 0.2}

    # dataset_list = ['gaoce']
elif target_region == 'padgroup':
    dataset_cur_thres = {
        'alltestval':{
            'binary accuracy': 0.99821,   
            'mclass accuracy': 0.99821,
            },
        'newval': {
            'binary accuracy': 0.99259,
            'mclass accuracy': 0.99259,
        },
        'bbtest':{
            'binary accuracy': 1.00000,
            'mclass accuracy': 1.00000,            
        },
        'oldval': {
            'binary accuracy': 0.99971,
            'mclass accuracy': 0.99971,    
        },
        'jiraissues':{
            'binary accuracy': 0.99965, 
            'mclass accuracy': 0.99965,  
        },
        'cur_jiraissue':{
            'binary accuracy': 1.00000,
            'mclass accuracy': 1.00000, 
        },
    }
    # dataset_list = ['newval', 'bbtest', 'oldval', 'masked', 'jiraissues'] #ori
    dataset_list = [
                    'alltestval', 'newval', 'bbtest', 'oldval',  'masked', 'jiraissues', 'graphite', 'unacceptable_error','body_box_select',
                      '3D', '3D_supply', 'cur_jiraissue'
                    ] # ori , 'ry'
    # dataset_list = ['jiraissues'] # ori , 'extreme_ng'

    lower_limit_range = 0.001
    test_dataset_cur_thres = {

        'masked':{
            'binary accuracy': 0.99710,
            'mclass accuracy': 0.99710,        
        },
        'graphite' :{
            'binary accuracy': 0.60000,
            'mclass accuracy': 0.60000,  
        },
        'unacceptable_error' :{
            'binary accuracy': 1.00000,
            'mclass accuracy': 1.00000,  
        },
        'body_box_select' :{
            'binary accuracy': 1.00000,
            'mclass accuracy': 1.00000,  
        },
        '3D_supply' :{
            'binary accuracy': 1.00000,
            'mclass accuracy': 1.00000,  
        },
        '3D' :{
            'binary accuracy': 0.99731,
            'mclass accuracy': 0.99731,  
        },
    }
    del_general_dataset_keys = []
    # dataset_list = ['alltestval', 'newval','oldval'] # ori 
    # valdataset_weights = {'alltestval': 0.8, 'newval': 1, 'bbtest': 1, 'oldval': 1, 'jiraissues': 0.5, 'masked': 1}
    
elif target_region == 'body':
    dataset_cur_thres = {
        'alltestval':{
            'binary accuracy': 0.98837,   
            'mclass accuracy': 0.98737,
            },
        'newval': {
            'binary accuracy': 0.99667,
            'mclass accuracy': 0.99556,
        },
        'bbtest':{
            'binary accuracy': 0.99846,
            'mclass accuracy': 0.99835,            
        },
        'oldval': {
            'binary accuracy': 0.99953,
            'mclass accuracy': 0.99953,    
        },
        'jiraissues':{
            'binary accuracy': 0.96869, 
            'mclass accuracy': 0.96483,  
        },
        'cur_jiraissue':{
            'binary accuracy': 0.73913,
            'mclass accuracy': 0.73913, 
        },
    }
    dataset_list = ['jiraissues', 'bbtestmz', 'bbtest', 'newval',  'alltestval', 'cur_jiraissue', 'gaoce_led', 'gaoce_trured_over', 'qb', 'dwzts'] #, 'led_cross_pair', 'melf', 'DA680'
    # dataset_list = ['bbtestmz'] #, 'led_cross_pair', 'melf', 'jlc'

    lower_limit_range = 0.003
    test_dataset_cur_thres = {
        'dwzts':{
            'binary accuracy': 0.22222,
            'mclass accuracy': 0.22222,
        },
        'qb':{
            'binary accuracy': 0.50000,
            'mclass accuracy': 0.50000,
        },
        'hty':{
            'binary accuracy': 0.75758,
            'mclass accuracy': 0.75758,        
        },
        'hty_tg':{
            'binary accuracy': 1.00000,
            'mclass accuracy': 1.00000,        
        },
        'bbtestmz':{
            'binary accuracy': 0.97864,
            'mclass accuracy': 0.97920,        
        },
        'cur_jiraissue':{
            'binary accuracy': 0.33333,
            'mclass accuracy': 0.33333,        
        },
        'gaoce_led': {
            'binary accuracy': 0.96464,
            'mclass accuracy': 0.95968,
        },
        'gaoce_trured_over': {
            'binary accuracy': 0.98246,
            'mclass accuracy': 0.99123,
        }
        }
    del_general_dataset_keys = ['oldval']

elif target_region == 'singlepad':
    dataset_cur_thres = {
        'alltestval':{
            'binary accuracy': 0.99808,   
            'mclass accuracy': 0.99772,
            },
        'newval': {
            'binary accuracy': 0.98498,
            'mclass accuracy': 0.98053,
        },
        'bbtest':{
            'binary accuracy': 0.99818,
            'mclass accuracy': 0.99763,            
        },
        'oldval': {
            'binary accuracy': 0.99953,
            'mclass accuracy': 0.99953,    
        },
        'jiraissues':{
            'binary accuracy': 0.99874, 
            'mclass accuracy': 0.99812,  
        },
        'cur_jiraissue':{
            'binary accuracy': 0.72414,
            'mclass accuracy': 0.72414, 
        },
    }
    dataset_list = ['bbtestmz', 'bbtest', 'jiraissues', 'newval', 'alltestval', 
                    '3D_old', '3D', 'cur_jiraissue', 'yl_cold_solder', 'yl_cold_solder_0512', 'yl_cold_solder_TG', 
                'bp_stove_clean', 'bp_stove_soft'] # 'newval', 'alltestval'
    # dataset_list = ['black_uuid'] # 'newval', 'alltestval', 'black_uuid'

    lower_limit_range = 0.003
    test_dataset_cur_thres = {
        'yl_cold_solder':{
            'binary accuracy': 1.00000,
            'mclass accuracy': 0.99455, 
        },
        'yl_cold_solder_0512':{
            'binary accuracy': 0.99673,
            'mclass accuracy': 0.69608, 
        },
        'alltestval_res':{
            'binary accuracy': 0.99980,   
            'mclass accuracy': 0.99978,
            },
        'black_uuid':{
            'binary accuracy': 0,
            'mclass accuracy': 0,        
        },
        'bbtestmz':{
            'binary accuracy': 0.97791,
            'mclass accuracy': 0.97349,        
        },
        '3D_old': {
            'binary accuracy': 0.99957,
            'mclass accuracy': 0.99957,
        },     
        '3D': {
            'binary accuracy': 0.95761,
            'mclass accuracy': 0.95012,
        },         
        'cur_jiraissue':{
            'binary accuracy': 0.99448,
            'mclass accuracy': 0.99448, 
        },
        'yl_cold_solder_TG':{
            'binary accuracy': 0.87574,
            'mclass accuracy': 0.86982, 
        },
        'bp_stove_clean':{
            'binary accuracy': 0.87574,
            'mclass accuracy': 0.86982, 
        },
        'bp_stove_soft':{
            'binary accuracy': 0.87574,
            'mclass accuracy': 0.86982, 
        },
    }
    
    del_general_dataset_keys = ['oldval', 'cur_jiraissue']
    
elif target_region == 'component':
    dataset_list = ['bbtestmz','bbtest', 'alltestval', 'jiraissues', 'tmpt']

for key in del_general_dataset_keys:
    dataset_cur_thres.pop(key, None) 

dataset_cur_thres.update(test_dataset_cur_thres)

targets = []

filter_targets = [


]

# body这部分，2.8.1是240913更新后的结果，md是模型反清的结果，bdg是用了生成的badcase的结果；
final_targets = [
]

version_compare = [
                        
                        # 'padgroupv1.22.5select',
                        'padgroupv1.22.6lut1select',


]

tag_dataset = ['alltestval', 'newval', 'bbtest', 'jiraissues', 'bbtestmz'] #'bbtestmz', 'jiraissues', 'bbtestmz', 
select_step1_select = False
final_targets_T = False  # True

select_step = 1
th_filter = 0.

while select_step:
    for dataset_i in dataset_list:
        if select_step == 2:
            print(f'================== ')
            print(f'== {dataset_i} === ')
            print(f'================== ')

        dataset_i_binary_accuracy = dataset_cur_thres[dataset_i]['binary accuracy'] - lower_limit_range
        dataset_i_mclass_accuracy = dataset_cur_thres[dataset_i]['mclass accuracy'] - lower_limit_range

        if select_step == 2:
            select_step1_select = True

        if select_step1_select:
            dataset_i_binary_accuracy = 0
            dataset_i_mclass_accuracy = 0

        compare_res_df_sub = compare_res_df_sorted
        for region_key, region_df in compare_res_df_sub.groupby(['region', 'version_folder']):
            region_df_sorted = region_df.sort_values(by=['final_score'], ascending=False)
            counter = 0
            counter_limit = 10
            for ii, model_anno in region_df_sorted.iterrows():
                if counter >= counter_limit:
                    continue

                chosen_model = model_anno['model_name']
                
                if select_step == 2:
                    if final_targets_T:
                        if chosen_model not in final_targets or chosen_model in filter_targets:
                            continue

                version_folder = model_anno['version_folder']
                if version_folder not in version_compare:
                    continue    

                chosen_model_res = all_res_df[(all_res_df['model_name'] == chosen_model) 
                                            & (all_res_df['val_dataset'] == dataset_i) 
                                            & (all_res_df['version_folder'] == region_key[1])].squeeze()
                
                show_dataset_title = True
                for th in list(threshold_dict.keys()):
                    if th == th_filter:
                        continue
                    # print(f'== {dataset_i} === ')
                    # print(chosen_model)
                    chosen_model_res_binary_accuracy = float(chosen_model_res[f'res_string-th{th}'].split('\n')[0].split(' ')[-1])
                    chosen_model_res_mclass_accuracy = float(chosen_model_res[f'res_string-th{th}'].split('\n')[1].split(' ')[-1])

                    if chosen_model_res_binary_accuracy >= dataset_i_binary_accuracy or chosen_model_res_mclass_accuracy >= dataset_i_mclass_accuracy:
                        if select_step == 1:
                            if dataset_i in tag_dataset:
                                final_targets.append(chosen_model)
                        elif select_step == 2:
                            if show_dataset_title:
                                print(f'------{version_folder}--------')
                                show_dataset_title = False

                            targets.append(chosen_model)
                            ckp_epoch = model_anno['ckp_epoch']
                            print(f'{chosen_model} ({ckp_epoch})')
                            print(f' ---- threshold = {th} ----')
                            print(chosen_model_res[f'res_string-th{th}'])
                counter += 1
    select_step += 1
    if select_step == 2:
        for chose_model_name in final_targets:
            print(chose_model_name)
    elif select_step > 2:
        break


== alltestval === 
------padgroupv1.22.6lut1select--------
padgroupfcdropoutmobilenetv3largers128128s42c2val0.1b256_ckp_bestv1.22.6slimmaskedf1certainlut1ed20.0j0.4lr0.025nb128nm128dual2top0.pth.tar (47)
 ---- threshold = 0.5 ----
binary accuracy = 0.98175
mclass accuracy = 0.98261
== Binary output ==
weighted f1 score = 0.9782082414330707
ok (n=5334): precision:0.993, recall:0.987, f1_score:0.992, omission_rate:0.080
ng (n=473): precision:0.865, recall:0.920, f1_score:0.902, omission_rate:0.013
== Multiclass output == 
weighted f1 score = nan
ok (n=5334): precision:0.993, recall:0.988, f1_score:0.992, omission_rate:0.080
solder_shortage (n=473): precision:0.873, recall:0.920, f1_score:0.905, omission_rate:0.012
ng (n=0): precision:0.873, recall:0.920, f1_score:0.905, omission_rate:0.012

padgroupfcdropoutmobilenetv3largers128128s42c2val0.1b256_ckp_bestv1.22.6slimmaskedf1certainlut1ed20.0j0.4lr0.025nb128nm128dual2top0.pth.tar (47)
 ---- threshold = 0.8 ----
binary accuracy = 0.98502
mc

In [1]:
# general datasets
from openpyxl import Workbook
from openpyxl.styles import Alignment

dataset_cur_thres = {
        'alltestval':{
            'binary accuracy': 0.98706,   
            'mclass accuracy': 0.98715,
            },
        'newval': {
            'binary accuracy': 0.99560,
            'mclass accuracy': 0.99560,
        },
        'bbtest':{
            'binary accuracy': 0.99895,
            'mclass accuracy': 0.99895,            
        },
        'oldval': {
            'binary accuracy': 0.99953,
            'mclass accuracy': 0.99953,    
        },
        'jiraissues':{
            'binary accuracy': 0.96583, 
            'mclass accuracy': 0.96399,  
        },
        'cur_jiraissue':{
            'binary accuracy': 0.72414,
            'mclass accuracy': 0.72414, 
        },
    }

##### ['bbtest', 'newval', 'alltestval']
if target_region == 'singlepinpad':
#     valdataset_weights = {'alltestval': 0.6, 'newval': 0.6, 'bbtest1': 0.5, 'bbtest2':1.0, 'jiraissues': 0.5}
    dataset_list = ['alltestval', 'newval', 'bbtest',  'bbtestmz', 'jiraissues',  'cur_jiraissue', 'cur_jiraissue_hrd_ks_TG'
                    ]
    lower_limit_range = 0.001
    test_dataset_cur_thres = {
        'bbtestmz':{
            'binary accuracy': 0.92495,
            'mclass accuracy': 0.91854,        
        },
        'cur_jiraissue_hrd_ks_TG':{
            'binary accuracy': 0.96970,
            'mclass accuracy': 0.96970, 
        },
    }
    del_general_dataset_keys = []

    # valdataset_weights = {'alltestval': 0.5, 'newval': 0.3, 'bbtest1': 0.3, 'bbtestmz':1.0, 'jiraissues': 0.2}

    # dataset_list = ['gaoce']
elif target_region == 'padgroup':
    # dataset_list = ['newval', 'bbtest', 'oldval', 'masked', 'jiraissues'] #ori
    dataset_list = [
                    'alltestval', 'newval', 'bbtest', 'oldval',  'masked', 'jiraissues', 'graphite', 'cur_jiraissue'
                    ] # ori 
    # dataset_list = ['jiraissues'] # ori 

    lower_limit_range = 0.001
    test_dataset_cur_thres = {
        'masked':{
            'binary accuracy': 0.99795,
            'mclass accuracy': 0.99795,        
        },
        'graphite' :{
            'binary accuracy': 0.20000,
            'mclass accuracy': 0.20000,  
        },
        'cur_jiraissues':{
            'binary accuracy': 0.66667,
            'mclass accuracy': 0.66667,              
        },


    }
    del_general_dataset_keys = []
    # dataset_list = ['alltestval', 'newval','oldval'] # ori 
    # valdataset_weights = {'alltestval': 0.8, 'newval': 1, 'bbtest': 1, 'oldval': 1, 'jiraissues': 0.5, 'masked': 1}
    
elif target_region == 'body':
    dataset_list = ['jiraissues', 'bbtestmz', 'bbtest', 'newval',  'alltestval', 'cur_jiraissue', 'gaoce_led', 'gaoce_trured_over'] #, 'led_cross_pair', 'melf', 'DA680'
    # dataset_list = ['bbtestmz'] #, 'led_cross_pair', 'melf'

    lower_limit_range = 0.003
    test_dataset_cur_thres = {
        'hty':{
            'binary accuracy': 0.75758,
            'mclass accuracy': 0.75758,        
        },
        'hty_tg':{
            'binary accuracy': 1.00000,
            'mclass accuracy': 1.00000,        
        },
        'bbtestmz':{
            'binary accuracy': 0.98355,
            'mclass accuracy': 0.98355,        
        },
        'cur_jiraissue':{
            'binary accuracy': 0.72414,
            'mclass accuracy': 0.72414,        
        },
        'gaoce_led': {
            'binary accuracy': 0.96149,
            'mclass accuracy': 0.96088,
        },
        'gaoce_trured_over': {
            'binary accuracy': 1.00000,
            'mclass accuracy': 1.00000,
        }
        }
    del_general_dataset_keys = ['oldval']

elif target_region == 'singlepad':
    dataset_list = ['bbtestmz', 'bbtest', 'jiraissues', 'newval', 'alltestval_res', '3D', 'cur_jiraissue'] # 'newval', 'alltestval'
    # dataset_list = ['black_uuid'] # 'newval', 'alltestval', 'black_uuid'

    lower_limit_range = 0.001
    test_dataset_cur_thres = {
        'alltestval_res':{
            'binary accuracy': 0.99980,   
            'mclass accuracy': 0.99978,
            },
        'black_uuid':{
            'binary accuracy': 0,
            'mclass accuracy': 0,        
        },
        'bbtestmz':{
            'binary accuracy': 0.97589,
            'mclass accuracy': 0.97172,        
        },
        '3D': {
            'binary accuracy': 0.99957,
            'mclass accuracy': 0.99957,
        },             
        'cur_jiraissue':{
            'binary accuracy': 0.98895,
            'mclass accuracy': 0.98895, 
        },
        'jsd':{
            'binary accuracy': 0.87574,
            'mclass accuracy': 0.86982, 
        },
    }
    
    del_general_dataset_keys = ['oldval', 'cur_jiraissue']
    
elif target_region == 'component':
    dataset_list = ['bbtestmz','bbtest', 'alltestval', 'jiraissues', 'tmpt']

for key in del_general_dataset_keys:
    dataset_cur_thres.pop(key, None) 

dataset_cur_thres.update(test_dataset_cur_thres)

targets = []

# body这部分，2.8.1是240913更新后的结果，md是模型反清的结果，bdg是用了生成的badcase的结果；
final_targets = [
]

version_compare = [
                # 'singlepadv0.13.0cp05nosp',                     
                # 'singlepadv0.13.0cp05sp',     
                #                 
                # 'singlepadv0.13.0lut05cp05nosp',                     
                # 'singlepadv0.13.0lut05cp05sp',   
                #                   
                # 'singlepadv0.13.0retraincp05nosp',                     
                # 'singlepadv0.13.0retraincp05sp', 
                #                     
                'singlepadv0.13.0retrainlut05cp05nosp',                     
                'singlepadv0.13.0retrainlut05cp05sp',

                # 'singlepadv0.13.1cp05nosp',
                # 'singlepadv0.13.1cp05sp',

                # 'singlepadv0.13.1lut05cp05nosp',
                # 'singlepadv0.13.1lut05cp05sp',

                # 'singlepadv0.13.1retraincp05nosp',
                # 'singlepadv0.13.1retraincp05sp',

                # 'singlepadv0.13.1retrainlut05cp05nosp',
                # 'singlepadv0.13.1retrainlut05cp05sp',       
]
tag_dataset = ['jiraissues', 'newval', 'bbtestmz'] #'bbtestmz', 
select_step1_select = False
final_targets_T = False
select_step = 2
th_filter = 0.5

wb = Workbook()
# 删除默认工作表
default_sheet = wb['Sheet']
wb.remove(default_sheet)
sheet_name = 'singlepadv0.13.0retrainlut05cp05'
ws = wb.create_sheet(title=sheet_name)

datasets_results = {}

while select_step:
    for dataset_i in dataset_list:
        if select_step == 2:
            print(f'================== ')
            print(f'== {dataset_i} === ')
            print(f'================== ')
        datasets_results[dataset_i] = {}
        dataset_i_binary_accuracy = dataset_cur_thres[dataset_i]['binary accuracy'] - lower_limit_range
        dataset_i_mclass_accuracy = dataset_cur_thres[dataset_i]['mclass accuracy'] - lower_limit_range

        if select_step == 2:
            select_step1_select = True

        if select_step1_select:
            dataset_i_binary_accuracy = 0
            dataset_i_mclass_accuracy = 0

        compare_res_df_sub = compare_res_df_sorted
        for region_key, region_df in compare_res_df_sub.groupby(['region', 'version_folder']):
            version_folder = region_key[1]
            if version_folder not in version_compare:
                continue  
           
            datasets_results[dataset_i][version_folder] = {}
            region_df_sorted = region_df.sort_values(by=['final_score'], ascending=False)
            counter = 0
            counter_limit = 10

            version_best_bi = -1
            version_best_bom = -1
            version_best_model_res = None
            for ii, model_anno in region_df_sorted.iterrows():
                if counter >= counter_limit:
                    continue

                chosen_model = model_anno['model_name']
                
                if select_step == 2:
                    if final_targets_T:
                        if chosen_model not in final_targets:
                            continue
                        
                chosen_model_res = all_res_df[(all_res_df['model_name'] == chosen_model) 
                                            & (all_res_df['val_dataset'] == dataset_i) 
                                            & (all_res_df['version_folder'] == region_key[1])].squeeze()
                
                show_dataset_title = True
                for th in list(threshold_dict.keys()):
                    if th == th_filter:
                        continue
                    # print(f'== {dataset_i} === ')
                    # print(chosen_model)
                    chosen_model_res_binary_accuracy = float(chosen_model_res[f'res_string-th{th}'].split('\n')[0].split(' ')[-1])
                    chosen_model_res_mclass_accuracy = float(chosen_model_res[f'res_string-th{th}'].split('\n')[1].split(' ')[-1])

                    if chosen_model_res_binary_accuracy > version_best_bi:
                        version_best_bi = chosen_model_res_binary_accuracy
                        version_best_bom = chosen_model_res_mclass_accuracy
                        version_best_model_res = chosen_model_res[f'res_string-th{th}'].split('\n')
                    elif chosen_model_res_binary_accuracy == version_best_bi:
                        if chosen_model_res_mclass_accuracy > version_best_bom:
                            version_best_bi = chosen_model_res_binary_accuracy
                            version_best_bom = chosen_model_res_mclass_accuracy
                            version_best_model_res = chosen_model_res[f'res_string-th{th}'].split('\n')
                    else:
                        continue
                counter += 1
            datasets_results[dataset_i][version_folder] = version_best_model_res

    select_step += 1
    if select_step == 2:
        for chose_model_name in final_targets:
            print(chose_model_name)
    elif select_step > 2:
        break

ws.cell(row=1, column=1, value = 'version_folder')

cols = 2
for dataset, infos in datasets_results.items():
    if ws.cell(row=1, column=cols).value is None:
        ws.cell(row=1, column=cols, value=dataset)

    version_rows = 0
    for version_folder, version_metrics in infos.items():
        version_metrics_infos = version_metrics[:-1]

        version_start_rows = version_rows * len(version_metrics_infos) + 2
        if ws.cell(row=version_start_rows, column=1).value is None:
            ws.cell(row=version_start_rows, column=1, value=version_folder)

        # 写入数据（从第二行开始）
        for i, metric in enumerate(version_metrics_infos):
            ws.cell(row=version_start_rows + i, column=cols, value=metric)
        version_rows += 1

    cols += 1

sample_dataset_infos = datasets_results[dataset]
merge_rows_start = 0
for version_folder, version_metrics in sample_dataset_infos.items(): 
    merge_start = merge_rows_start * len(version_metrics_infos) + 2
    merge_end = merge_start + len(version_metrics_infos) - 1
    ws.merge_cells(start_row=merge_start, start_column=1, end_row=merge_end, end_column=1)
    merged_cell = ws.cell(row=merge_start, column=1, value=version_folder)
    merged_cell.alignment = Alignment(horizontal='center', vertical='center')
    merge_rows_start += 1
wb.save(f"{sheet_name}.xlsx")

NameError: name 'target_region' is not defined